# 克漏字干擾選項生成

## 測試資料

In [2]:
text1 = "To make Jane live a [MASK] life, Mother was very careful about spending money. [SEP] happy"
# "happy" -> "poor", "busy", "sad"
text2 = "[MASK] , Jane didn't understand her. [SEP] However"
# "However" -> "Though", "Although", "Or"
text3 = "Every day Mother was busy with her [MASK] while Jane was studying at school, so they had little time to enjoy themselves. [SEP] work"
# "work" -> "writing", "housework", "research"
text4 = "One day, Mother realized Jane was unhappy and even [MASK] to her. [SEP] unfriendly"
# "unfriendly" -> "loyal", "kind", "cruel"
text5 = "The wait seemed [MASK] . [SEP] endless"
# "endless" -> "meaningless", "useless", "careless"
text6 = "The old man was waiting for a ride across the [MASK] . [SEP] river"
# "river" -> "town", "country", "island"
text7 = "I felt uncomfortable and out of place as the professor carefully [MASK] what she expected us to learn. [SEP] explained"
# "explained" -> "showed", "designed", "offered"
text8 = "As I listened, I couldn't help but [MASK] of my own oldest daughter. [SEP] think"
# "think" -> "speak", "talk", "hear"
text9 = "As I listened, I couldn't help but think of my own oldest daughter who was now beginning her first year in [MASK], just like me. [SEP] college"
# "college" -> "school", "office", "classroom"

# Sentence on internet
text10 = "Today morning, I saw a [MASK] sitting on the wall. [SEP] cat"
text11 = "Ukrainian presidential adviser says situation is ' [MASK] control' in suburbs and outskirts of Kyiv. [SEP] under"
text12 = "I don't think that after what is [MASK] now, Ukraine has weak positions. [SEP] happening"

# Test data of DGen
text13 = "If an object is attracted to a magnet, the object is most likely made of [MASK]. [SEP] metal"
# "metal" -> "wood", "plastic", "cardboard"
text14 = "the main organs of the respiratory system are [MASK]. [SEP] lungs"
# "lungs" -> "ovaries", "intestines", "kidneys"
text15 = "The products of photosynthesis are glucose and [MASK] else. [SEP] oxygen"
# "oxygen" -> "carbon", "hydrogen", "nitrogen"
text16 = "frogs have [MASK] eyelid membranes. [SEP] three"
# "three" -> "two", "four", "one"
text17 = "the only known planet with large amounts of water is [MASK]. [SEP] earth"
# "earth" -> "saturn", "jupiter", "mars"

## Demo questions

* q1 ~ q10 取自CLOTH資料集，名詞2個，動詞3個，形容詞3個，副詞2個
* q11 ~ q20 取自DGen資料集，名詞8個，形容詞1個，數量詞1個
* q21 ~ q23 取自網路文章

In [14]:
questions = {
    "q1": {
        "sentence": "To make Jane live a [MASK] life, Mother was very careful about spending money. [SEP] happy",
        "answer": "happy",
        "distractors": ["poor", "busy", "sad"]
    },
    "q2": {
        "sentence": "[MASK] , Jane didn't understand her. [SEP] However",
        "answer": "However",
        "distractors": ["Though", "Although", "Or"]
    },
    "q3": {
        "sentence": "Every day Mother was busy with her [MASK] while Jane was studying at school, so they had little time to enjoy themselves. [SEP] work",
        "answer": "work",
        "distractors": ["writing", "housework", "research"]
    },
    "q4": {
        "sentence": "One day, Mother realized Jane was unhappy and even [MASK] to her. [SEP] unfriendly",
        "answer": "unfriendly",
        "distractors": ["loyal", "kind", "cruel"]
    },
    "q5": {
        "sentence": "The old man was waiting for a ride across the [MASK] . [SEP] river",
        "answer": "river",
        "distractors": ["town", "country", "island"]
    },
    "q6": {
        "sentence": "I felt uncomfortable and out of place as the professor carefully [MASK] what she expected us to learn. [SEP] explained",
        "answer": "explained",
        "distractors": ["showed", "designed", "offered"]
    },
    "q7": {
        "sentence": "As I listened, I couldn't help but [MASK] of my own oldest daughter. [SEP] think",
        "answer": "think",
        "distractors": ["speak", "talk", "hear"]
    },
    "q8": {
        "sentence": "As we were [MASK] on the third floor for old people with Alzheimer, most of them stared off at the walls or floor. [SEP] singing",
        "answer": "singing",
        "distractors": ["meeting", "gathering", "dancing"]
    },
    "q9": {
        "sentence": "As we got [MASK] with each song, she did as well. [SEP] louder",
        "answer": "louder",
        "distractors": ["higher", "nearer", "faster"]
    },
    "q10": {
        "sentence": "Mr. Petri,  _   injured in the fire, was rushed to hospital. [SEP] seriously",
        "answer": "seriously",
        "distractors": ["blindly", "hardly", "slightly"]
    },
    "q11": {
        "sentence": "If an object is attracted to a magnet, the object is most likely made of [MASK]. [SEP] metal",
        "answer": "metal",
        "distractors": ["wood", "plastic", "cardboard"]
    },
    "q12": {
        "sentence": "the main organs of the respiratory system are [MASK]. [SEP] lungs",
        "answer": "lungs",
        "distractors": ["ovaries", "intestines", "kidneys"]
    },
    "q13": {
        "sentence": "The products of photosynthesis are glucose and [MASK] else. [SEP] oxygen",
        "answer": "oxygen",
        "distractors": ["carbon", "hydrogen", "nitrogen"]
    },
    "q14": {
        "sentence": "frogs have [MASK] eyelid membranes. [SEP] three",
        "answer": "three",
        "distractors": ["two", "four", "one"]
    },
    "q15": {
        "sentence": "the only known planet with large amounts of water is [MASK]. [SEP] earth",
        "answer": "earth",
        "distractors": ["saturn", "jupiter", "mars"]
    },
    "q16": {
        "sentence": "[MASK] is responsible for erosion by flowing water and glaciers. [SEP] gravity",
        "answer": "gravity",
        "distractors": ["kinetic", "electromagnetic", "weight"],
    },
    "q17": {
        "sentence": "Common among mammals and insects , pheromones are often related to [MASK] type of behavior. [SEP] reproductive",
        "answer": "reproductive",
        "distractors": ["aggressive", "immune", "cardiac"]
    },
    "q18": {
        "sentence": "[MASK] can reproduce by infecting the cell of a living host. [SEP] virus",
        "answer": "virus",
        "distractors": ["bacteria", "mucus", "carcinogens"]
    },
    "q19": {
        "sentence": "proteins are encoded by [MASK]. [SEP] genes",
        "answer": "genes",
        "distractors": ["DNA", "RNA", "codons"]
    },
    "q20": {
        "sentence": "Producers at the base of ecological food webs are also known as [MASK]. [SEP] autotrophic",
        "answer": "autotrophic",
        "distractors": ["endoscopic", "symbiotic", "mutualistic"],
    },
    "q21": {
        "sentence": "Today morning, I saw a [MASK] sitting on the wall. [SEP] cat",
        "answer": "cat",
        "distractors": [],
    },
    "q22": {
        "sentence": "Ukrainian presidential adviser says situation is ' [MASK] control' in suburbs and outskirts of Kyiv. [SEP] under",
        "answer": "under",
        "distractors": [],
    },
    "q23": {
        "sentence": "I don't think that after what is [MASK] now, Ukraine has weak positions. [SEP] happening",
        "answer": "happening",
        "distractors": [],
    },
}

## Candidate Set Generator (CSG)

使用BERT預訓練模型，生成CS。

### 引用相關套件

In [3]:
from tqdm.notebook import tqdm
import os

### 載入CSG模型

* BERT_model: BERT + 3 distractors

In [4]:
from transformers import BertTokenizer, BertConfig, BertForMaskedLM, pipeline

dir_path = r"./models/CSG/BERT_model"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
config = BertConfig.from_pretrained(os.path.join(dir_path, "config.json"))
csg_model = BertForMaskedLM.from_pretrained(os.path.join(dir_path, "pytorch_model.bin"), from_tf=bool('.ckpt' in 'bert-base-uncased'), config=config)
csg_model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [5]:
my_unmasker = pipeline('fill-mask', tokenizer=tokenizer, config=config, model=csg_model, top_k=10)

In [15]:
my_unmasker(questions["q1"]["sentence"])

[{'score': 0.17808528244495392,
  'token': 6517,
  'token_str': 's a d',
  'sequence': 'to make jane live a sad life, mother was very careful about spending money. happy'},
 {'score': 0.05460711568593979,
  'token': 11771,
  'token_str': 'b o r i n g',
  'sequence': 'to make jane live a boring life, mother was very careful about spending money. happy'},
 {'score': 0.04597169905900955,
  'token': 5697,
  'token_str': 'b u s y',
  'sequence': 'to make jane live a busy life, mother was very careful about spending money. happy'},
 {'score': 0.02937108650803566,
  'token': 3407,
  'token_str': 'h a p p y',
  'sequence': 'to make jane live a happy life, mother was very careful about spending money. happy'},
 {'score': 0.019477127119898796,
  'token': 2919,
  'token_str': 'b a d',
  'sequence': 'to make jane live a bad life, mother was very careful about spending money. happy'},
 {'score': 0.01932520419359207,
  'token': 4138,
  'token_str': 'r i c h',
  'sequence': 'to make jane live a rich 

## Candidate Set (CS)

* cs = 候選集合(單字&分數)
* target_sent = 目標句子
* origin_sent = 原始句子
* answer = 答案

In [16]:
target_sent = questions["q1"]["sentence"]
print(target_sent)

To make Jane live a [MASK] life, Mother was very careful about spending money. [SEP] happy


In [17]:
origin_sent = target_sent.split(" [SEP] ")[0]
print(origin_sent)

To make Jane live a [MASK] life, Mother was very careful about spending money.


In [18]:
answer = questions["q1"]["answer"]
print(answer)

happy


In [19]:
cs = list()

for cand in my_unmasker(target_sent):
    cs.append({"word": cand["token_str"].replace(" ", ""), "s0": cand["score"], "s1": 0.0, "s2": 0.0, "s3": 0.0})

print(cs)

[{'word': 'sad', 's0': 0.17808528244495392, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'boring', 's0': 0.05460711568593979, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'busy', 's0': 0.04597169905900955, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'happy', 's0': 0.02937108650803566, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'bad', 's0': 0.019477127119898796, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'rich', 's0': 0.01932520419359207, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'lazy', 's0': 0.017796039581298828, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'normal', 's0': 0.01709771901369095, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'difficult', 's0': 0.01621469482779503, 's1': 0.0, 's2': 0.0, 's3': 0.0}, {'word': 'strange', 's0': 0.01619175635278225, 's1': 0.0, 's2': 0.0, 's3': 0.0}]


## Distractor Selector (DS)

四種評分標準：

1. 單字相似度：和答案越接近，分數越低
2. 句子相似度：和答案的句子越接近，分數越低
3. 詞性相似度：和答案詞性相同，分數越高
4. 型態相似度(skip!)：字串長度相同、字首字尾相同，分數越高

### 載入fastText模型

In [107]:
import fasttext
import warnings

model_path = r"./models/DS/fasttext_model/wiki_en_ft_model01.bin"

ds_model = fasttext.load_model(model_path)
warnings.filterwarnings("ignore", category=FutureWarning)

### 兩向量距離公式(cosine similarity)

In [14]:
import numpy as np

def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

### 資料標準化

#### Min-Max

In [15]:
# Min-Max 歸一化
def min_max_y(raw_data):
  # 裝進標準化後的新串列
  min_max_data = []
  
  # 進行Min-Max標準化
  for d in raw_data:
    min_max_data.append((d - min(raw_data)) / (max(raw_data) - min(raw_data)))
             
  # 回傳結果
  return min_max_data

#### Z-Score

In [35]:
import math

# Z-Score標準化
def z_score_y(raw_data):
  print(raw_data)
  # 計算數據平均數
  total = 0
  for d in raw_data:
    total += d
  aver = total/len(raw_data)
  
  # 計算所有數據減掉平均數的平方相加
  data_s = 0
  for s in raw_data:
    data_s += (s - aver)**2
  
  # 計算剛剛的結果除以數據總數開根號即為標準差
  std = math.sqrt(data_s/len(raw_data))
  
  # 將每個數據減掉平均數除以標準差成為新的數據串列
  # 裝新的標準化後的數據串列
  z_score_data = []
  for z in raw_data:
    z_score_data.append((z - aver)/std)
  
  # 回傳結果
  return z_score_data

### 0.模型信心分數

In [177]:
s0s = [c["s0"] for c in cs]
new_s0s = min_max_y(s0s)

for i, c in enumerate(cs):
    print(c["word"], new_s0s[i])
    c["s0"] = new_s0s[i]

water 1.0
something 0.19788846252551132
everything 0.1699093879187814
sugar 0.11799695081049234
oxygen 0.11676989337155907
energy 0.08610428941358418
gas 0.07822988459581895
anything 0.06374510726952969
air 0.05685093315061298
food 0.05004736045113694
whatever 0.041838423265378864
nothing 0.029247714734558214
blood 0.025990574049384364
oil 0.02162749372239562
coffee 0.018868922342895948
juice 0.017731083363568396
meat 0.010010257288489007
carbon 0.007053145176441214
somebody 0.002499265532934739
heat 0.0021308804844136237
ice 0.0013396879343234832
waste 0.0012960588860787669
everybody 0.0011324247266742446
alcohol 0.0010147573090425448
power 0.000983350256090421
sunlight 0.0006452604520754421
chemical 0.0005639755837563628
coke 0.0003630085289089874
salt 0.0003442437997779709
rice 0.0


### 1.單字相似度

In [178]:
print(answer)
print("-"*100)
answer_vector = ds_model.get_word_vector(answer)

word_similarities = list()
for c in cs:
  c_vector = ds_model.get_word_vector(c["word"])
  word_similarity = similarity(answer_vector, c_vector)
  word_similarities.append(word_similarity)

new_similarities = min_max_y(word_similarities)

for i, c in enumerate(cs):
    print(c["word"], 1-word_similarities[i], 1-new_similarities[i])
    c["s2"] = 1-new_similarities[i]

oxygen
----------------------------------------------------------------------------------------------------
water 0.4342515468597412 0.4624379277229309
something 0.7874229550361633 0.8385330587625504
everything 0.7785428762435913 0.8290765881538391
sugar 0.8342749625444412 0.8884261474013329
oxygen 1.1920928955078125e-07 0.0
energy 0.4840930104255676 0.5155144929885864
gas 0.4517453908920288 0.4810672402381897
anything 0.774000808596611 0.8242397010326385
air 0.650581032037735 0.6928089559078217
food 0.6804319322109222 0.7245974242687225
whatever 0.8088977187871933 0.8614017069339752
nothing 0.8129235655069351 0.8656888753175735
blood 0.5433814525604248 0.5786512494087219
oil 0.660214900970459 0.7030681669712067
coffee 0.8201776742935181 0.8734138309955597
juice 0.7489938735961914 0.7976096123456955
meat 0.7934980541467667 0.8450024873018265
carbon 0.36206382513046265 0.3855646252632141
somebody 0.8601741343736649 0.9160063862800598
heat 0.4419863224029541 0.4706747531890869
ice 0.6356

### 2.句子相似度

* correct_sent = 填入正確答案句子
* cand_sents = 填入候選詞句子

In [179]:
#依據訓練過後的BERT所生成選項放入句子做比較
correct_sent = origin_sent.replace('[MASK]', answer)
print(correct_sent)
print("-"*100)
correct_sent_vector = ds_model.get_sentence_vector(correct_sent)

cand_sents = list()
for c in cs:
  cand_sents.append(origin_sent.replace('[MASK]', c["word"]))

sent_similarities = list()
#兩句子距離
for cand_sent in cand_sents:
  cand_sent_vector = ds_model.get_sentence_vector(cand_sent)
  sent_similarity = similarity(correct_sent_vector, cand_sent_vector)
  sent_similarities.append(sent_similarity)

new_similarities = min_max_y(sent_similarities)
for i, c in enumerate(cs):
    print(cand_sents[i], 1-sent_similarities[i], 1-new_similarities[i])
    c["s2"] = 1-new_similarities[i]

The products of photosynthesis are glucose and oxygen else.
----------------------------------------------------------------------------------------------------
The products of photosynthesis are glucose and water else. 0.016998887062072754 0.4499818682670593
The products of photosynthesis are glucose and something else. 0.03024733066558838 0.8006847649812698
The products of photosynthesis are glucose and everything else. 0.0298154354095459 0.7892519533634186
The products of photosynthesis are glucose and sugar else. 0.03344142436981201 0.8852364346385002
The products of photosynthesis are glucose and oxygen else. 0.0 0.0
The products of photosynthesis are glucose and energy else. 0.019023478031158447 0.503575325012207
The products of photosynthesis are glucose and gas else. 0.017865657806396484 0.47292637825012207
The products of photosynthesis are glucose and anything else. 0.029795825481414795 0.7887328565120697
The products of photosynthesis are glucose and air else. 0.025262117385

### 3.詞性相似度

In [125]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andy1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\andy1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [180]:
origin_token = word_tokenize(origin_sent)
origin_token.remove("[")
origin_token.remove("]")
# print(origin_token)

mask_index = origin_token.index("MASK")
# print(mask_index)

correct_token = word_tokenize(correct_sent)
# print(correct_token)
correct_pos = nltk.pos_tag(correct_token)
# print(correct_pos)

answer_pos = correct_pos[mask_index]
print(answer_pos)
print("-"*100)

for i, c in enumerate(cs):
    cand_sent_token = word_tokenize(cand_sents[i])
    cand_sent_pos = nltk.pos_tag(cand_sent_token)
    cand_pos = cand_sent_pos[mask_index]

    if cand_pos[1] == answer_pos[1]:
        c["s3"] = 1.0
    else:
        c["s3"] = 0.0
    
    print(cand_pos, c["s3"])

('oxygen', 'NN')
----------------------------------------------------------------------------------------------------
('water', 'NN') 1.0
('something', 'NN') 1.0
('everything', 'NN') 1.0
('sugar', 'NN') 1.0
('oxygen', 'NN') 1.0
('energy', 'NN') 1.0
('gas', 'NN') 1.0
('anything', 'NN') 1.0
('air', 'NN') 1.0
('food', 'NN') 1.0
('whatever', 'WDT') 0.0
('nothing', 'NN') 1.0
('blood', 'NN') 1.0
('oil', 'NN') 1.0
('coffee', 'NN') 1.0
('juice', 'NN') 1.0
('meat', 'NN') 1.0
('carbon', 'NN') 1.0
('somebody', 'NN') 1.0
('heat', 'NN') 1.0
('ice', 'NN') 1.0
('waste', 'NN') 1.0
('everybody', 'NN') 1.0
('alcohol', 'NN') 1.0
('power', 'NN') 1.0
('sunlight', 'JJ') 0.0
('chemical', 'JJ') 0.0
('coke', 'VBD') 0.0
('salt', 'JJ') 0.0
('rice', 'NN') 1.0


### 4.型態相似度(skip!)

In [ ]:
ans_len = len(answer)
for c in cs:
    dis_len = len(c)
    diff = max(1 - (abs(dis_len-ans_len) / ans_len), 0)
    print(c, diff)

sad 0.6
boring 0.8
busy 0.8
happy 1.0
bad 0.6
rich 0.8
lazy 0.8
normal 0.8
difficult 0.19999999999999996
strange 0.6


## Ranker

In [181]:
cs

[{'word': 'water', 's0': 1.0, 's1': 0.0, 's2': 0.4499818682670593, 's3': 1.0},
 {'word': 'something',
  's0': 0.19788846252551132,
  's1': 0.0,
  's2': 0.8006847649812698,
  's3': 1.0},
 {'word': 'everything',
  's0': 0.1699093879187814,
  's1': 0.0,
  's2': 0.7892519533634186,
  's3': 1.0},
 {'word': 'sugar',
  's0': 0.11799695081049234,
  's1': 0.0,
  's2': 0.8852364346385002,
  's3': 1.0},
 {'word': 'oxygen',
  's0': 0.11676989337155907,
  's1': 0.0,
  's2': 0.0,
  's3': 1.0},
 {'word': 'energy',
  's0': 0.08610428941358418,
  's1': 0.0,
  's2': 0.503575325012207,
  's3': 1.0},
 {'word': 'gas',
  's0': 0.07822988459581895,
  's1': 0.0,
  's2': 0.47292637825012207,
  's3': 1.0},
 {'word': 'anything',
  's0': 0.06374510726952969,
  's1': 0.0,
  's2': 0.7887328565120697,
  's3': 1.0},
 {'word': 'air',
  's0': 0.05685093315061298,
  's1': 0.0,
  's2': 0.6687199175357819,
  's3': 1.0},
 {'word': 'food',
  's0': 0.05004736045113694,
  's1': 0.0,
  's2': 0.7126792669296265,
  's3': 1.0},
 

### 加上權重 (final score)

In [182]:
cs_fs = list()
for c in cs:
    fs = 0.25*c["s0"] + 0.25*c["s1"] + 0.25*c["s2"] + 0.25*c["s3"]
    cs_fs.append((c["word"], fs))

cs_fs

[('water', 0.6124954670667648),
 ('something', 0.4996433068766953),
 ('everything', 0.48979033532055),
 ('sugar', 0.5008083463622481),
 ('oxygen', 0.27919247334288977),
 ('energy', 0.3974199036064478),
 ('gas', 0.38778906571148525),
 ('anything', 0.46311949094539984),
 ('air', 0.43139271267159873),
 ('food', 0.44068165684519084),
 ('whatever', 0.2157310696575446),
 ('nothing', 0.465952802115799),
 ('blood', 0.39851865879879167),
 ('oil', 0.4293446113723408),
 ('coffee', 0.4705675130461751),
 ('juice', 0.4533310639188378),
 ('meat', 0.46294450758016936),
 ('carbon', 0.3431847414389925),
 ('somebody', 0.47900093484949174),
 ('heat', 0.3667293376051961),
 ('ice', 0.418021376935344),
 ('waste', 0.40112080812276096),
 ('everybody', 0.4814061903840774),
 ('alcohol', 0.3977528632068841),
 ('power', 0.43450126617625484),
 ('sunlight', 0.1336262624558915),
 ('chemical', 0.11593092888872931),
 ('coke', 0.19758045706897004),
 ('salt', 0.16528115350880526),
 ('rice', 0.5)]

### 排序

In [183]:
cs_fs.sort(key = lambda x: x[1], reverse=True)
cs_fs

[('water', 0.6124954670667648),
 ('sugar', 0.5008083463622481),
 ('rice', 0.5),
 ('something', 0.4996433068766953),
 ('everything', 0.48979033532055),
 ('everybody', 0.4814061903840774),
 ('somebody', 0.47900093484949174),
 ('coffee', 0.4705675130461751),
 ('nothing', 0.465952802115799),
 ('anything', 0.46311949094539984),
 ('meat', 0.46294450758016936),
 ('juice', 0.4533310639188378),
 ('food', 0.44068165684519084),
 ('power', 0.43450126617625484),
 ('air', 0.43139271267159873),
 ('oil', 0.4293446113723408),
 ('ice', 0.418021376935344),
 ('waste', 0.40112080812276096),
 ('blood', 0.39851865879879167),
 ('alcohol', 0.3977528632068841),
 ('energy', 0.3974199036064478),
 ('gas', 0.38778906571148525),
 ('heat', 0.3667293376051961),
 ('carbon', 0.3431847414389925),
 ('oxygen', 0.27919247334288977),
 ('whatever', 0.2157310696575446),
 ('coke', 0.19758045706897004),
 ('salt', 0.16528115350880526),
 ('sunlight', 0.1336262624558915),
 ('chemical', 0.11593092888872931)]

## Distractors

### Top 3

In [184]:
distractors = cs_fs[:3]
distractors

[('water', 0.6124954670667648), ('sugar', 0.5008083463622481), ('rice', 0.5)]

In [185]:
result = [d[0] for d in distractors]
result

['water', 'sugar', 'rice']

## Result

### text13

"If an object is attracted to a magnet, the object is most likely made of [MASK]. [SEP] metal"

ans: metal  
dis: wood, plastic, cardboard  
result: **wood**, glass, water

### text14

"the main organs of the respiratory system are [MASK]. [SEP] lungs"

ans: lungs  
dis: ovaries, intestines, kidneys  
result: eyes, organs, ears

### text15

"The products of photosynthesis are glucose and [MASK] else. [SEP] oxygen"

ans: oxygen  
dis: carbon, hydrogen, nitrogen  
result: water, sugar, something  
result(30): water, sugar, rice

### text16

"frogs have [MASK] eyelid membranes. [SEP] three"

ans: three  
dis: two, four, one  
result: **four**, **one**, **two**

### text17

"the only known planet with large amounts of water is [MASK]. [SEP] earth"

ans: earth  
dis: saturn, jupiter, mars  
result: moon, *earth*, sea  
result(30): moon, asia, africa



## Record

* top30好像沒有比較好